<a href="https://colab.research.google.com/github/Denzzo-ov/Simulative_Python_Final_project/blob/main/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подготовим окружение

In [47]:
# Общее
import os # работа с файловой системой
import logging # логирование
from collections import Counter # подсчет элементов в коллекции

# Дата/время
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import time

# Работа с данными
import requests # обращение к API
import ast
import psycopg2 # подключение к базе данных

# Google API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials

Настроим логи

In [46]:
# Создаем дирректорию для логов
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Создаем файла для логов
moscow_time = datetime.now(ZoneInfo("Europe/Moscow"))
log_file = f"logs/{moscow_time:%Y-%m-%d}.log"

# Настраиваем логирования через FileHandler
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Очищаем старые обработчики, если есть
logger.handlers.clear()

file_handler = logging.FileHandler(log_file, encoding='utf-8')
formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Корректируем время в логах по Москве
logging.Formatter.converter = lambda *args: moscow_time.timetuple()

logging.info("Логирование инициализировано. Файл: %s", log_file)

# Удаляем лог-файлы старше 3 дней
logging.info("Запуск очистки старых логов")
for filename in os.listdir(log_dir):
    if filename.endswith(".log"):
        try:
            file_date = datetime.strptime(filename.replace(".log", ""), "%Y-%m-%d")
            if datetime.now() - file_date > timedelta(days=3):
                os.remove(os.path.join(log_dir, filename))
        except Exception as e:
            print(f"Не удалось обработать лог-файл {filename}: {e}")

logging.info("Очистка старых логов завершена")

Запрос к API

In [48]:
# адрес API
api_url = "https://b2b.itresume.ru/api/statistics"

# параметры подключения
params = {
    'client':'Skillfactory',
    'client_key':'M2MGWS',
    'start':'2025-12-31 00:00:00',
    'end': '2026-01-13 23:59:59'
    }

logging.info("Начинаем загрузку данных с API")

try:
    response = requests.get(api_url, params=params)
    response.raise_for_status()
    data = response.json()
    logger.info(f"Данные успешно получены, записей: {len(data)}")
except requests.RequestException as e:
    logging.error(f"Ошибка при запросе к API: {e}")
    data = []

Сохранение полученных данных

In [49]:
processed_data = []

# Получение параметра passback_params
for item in data:
    raw_passback = item.get("passback_params")

    # Проверка валидности passback_params
    if not raw_passback or not isinstance(raw_passback, str) or raw_passback.strip() == "":
        logging.error(f"passback_params отсутствует или не строка, запись пропущена (user_id={item.get('lti_user_id')})")
        continue   # Пропуск текущей записи при ошибке

    # Преобразование строки в словарь с проверкой на ошибки
    try:
        passback = ast.literal_eval(raw_passback)

        # Обработка поля is_correct
        raw_is_correct = item.get("is_correct")
        if raw_is_correct is None:
            is_correct = None
        elif raw_is_correct in (1, True):
            is_correct = True
        elif raw_is_correct in (0, False):
            is_correct = False
        else:
            logging.error(f"Некорректное поле is_correct: {raw_is_correct}")
            continue

        # Формирование обработанной записи
        processed_data.append({
            "user_id": item.get("lti_user_id"),
            "oauth_consumer_key": passback.get("oauth_consumer_key"),
            "lis_result_sourcedid": passback.get("lis_result_sourcedid"),
            "lis_outcome_service_url": passback.get("lis_outcome_service_url"),
            "is_correct": is_correct,
            "attempt_type": item.get("attempt_type"),
            "created_at": item.get("created_at")
        })

    except (ValueError, SyntaxError) as e:
        logging.error(f"Некорректный passback_params: {e}, пропуск записи")
        continue
    except Exception as e:
        logging.error(f"Ошибка при обработке записи: {e}")

# Фиксация результата предобработки
logging.info(f"Предобработка данных завершена, подготовлено {len(processed_data)} записей")


Запись в облачную базу данных

In [50]:
# подключение к базе данных
try:
    conn = psycopg2.connect(
        dbname="homework",
        user="admin_user",
        password="Simulative123",
        host="109.248.201.217",
        port="5432"
    )
    cursor = conn.cursor()
    logging.info("Подключение к базе данных установлено")

    # Создание таблицы
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS grader_try (
            user_id TEXT,
            oauth_consumer_key TEXT,
            lis_result_sourcedid TEXT,
            lis_outcome_service_url TEXT,
            is_correct BOOLEAN,
            attempt_type TEXT,
            created_at TIMESTAMP
        )
    """)
    conn.commit()

    # Подготовка данных
    insert_query = """
        INSERT INTO grader_try
        (user_id, oauth_consumer_key, lis_result_sourcedid,
         lis_outcome_service_url, is_correct, attempt_type, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    data_to_insert = [
        (
            record['user_id'],
            record['oauth_consumer_key'],
            record['lis_result_sourcedid'],
            record['lis_outcome_service_url'],
            record['is_correct'],
            record['attempt_type'],
            record['created_at']
        )
        for record in processed_data
    ]

    # Вставка данных в базу(execute_batch)
    from psycopg2.extras import execute_batch
    execute_batch(cursor, insert_query, data_to_insert, page_size=1000)
    conn.commit()
    logging.info("Данные успешно загружены")

except Exception as e:
    logging.error(f"Ошибка работы с базой данных: {e}")
    conn.rollback()  # откат при ошибке

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
    logging.info("Соединение с базой данных закрыто")

Загрузка отчет в Google Sheets

In [51]:
# Агрегация статистики
total_attempts = len(processed_data)
successful_attempts = sum(1 for x in processed_data if x.get("is_correct"))
unique_users = len(set(x["user_id"] for x in processed_data if "user_id" in x))
attempts_per_type = Counter(x.get("attempt_type", "unknown") for x in processed_data)

daily_stats = {
    "unique_users": unique_users,                         # Количество уникальных пользователей
    "date": moscow_time.strftime("%Y-%m-%d"),             # Дата отчета (по Москве)
    "total_attempts": total_attempts,                     # Всего попыток
    "successful_attempts": successful_attempts,           # Успешных попыток
    "run_attempts": attempts_per_type.get("run", 0),      # Попыток "run"
    "submit_attempts": attempts_per_type.get("submit", 0) # Попыток "submit"
}

logging.info(f"Расчет ежедневной статистики: {daily_stats}")

try:
    # Аутентификация (исправленный scope)
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive.file"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        '/content/drive/MyDrive/Service_account/service_account_key.json',
        scopes=scope
    )
    gc = gspread.authorize(creds)
    logging.info("Аутентификация в Google Sheets успешна")

    # Открытие таблицы и листа
    sheet_id = "1flatOTqLwAzEuRmghad-RmE5IjFQHHLijdOZu-p-L_Q"
    spreadsheet = gc.open_by_key(sheet_id)

    try:
        sheet = spreadsheet.worksheet("Stat")
    except gspread.exceptions.WorksheetNotFound:
        logging.warning("Лист 'Stat' не найден. Создаю новый...")
        sheet = spreadsheet.add_worksheet("Stat", rows="1000", cols="20")

    # Добавление заголовков (если их нет)
    first_row = sheet.row_values(1)
    if not first_row:
        headers = ["Date",
                   "Unique Users",
                   "Total Attempts",
                   "Successful Attempts",
                   "Run Attempts",
                   "Submit Attempts"
                   ]
        sheet.insert_row(headers, 1)
        logging.info("Заголовки столбцов добавлены в таблицу")
    else:
        logging.info("Заголовки уже присутствуют в таблице")

    # Подготовка и отправка данных
    sheet_row = [
        daily_stats["date"],                # Дата отчета
        daily_stats["unique_users"],        # Количество уникальных пользователей
        daily_stats["total_attempts"],      # Всего попыток
        daily_stats["successful_attempts"], # Успешных попыток
        daily_stats["run_attempts"],        # Попыток "run"
        daily_stats["submit_attempts"]      # Попыток "submit"
    ]

    logging.info(f"Отправляю в таблицу: {sheet_row}")
    sheet.append_row(sheet_row)
    logging.info("Данные успешно загружены в Google Sheets")

except gspread.exceptions.SpreadsheetNotFound:
    logging.error("Таблица не найдена. Проверьте ID и права доступа.")
except gspread.exceptions.APIError as e:
    logging.error(f"Ошибка API Google Sheets: {e}")
except FileNotFoundError:
    logging.error("Файл сервисного аккаунта не найден. Проверьте путь.")
except Exception as e:
    logging.error(f"Неожиданная ошибка: {e}")